In [1]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
from dataUtil import getBalanceData, getBudgetingData, getHousingData
import numpy as np
import tflearn
import tensorflow as tf
import random
import json

In [2]:
stemmer = LancasterStemmer()

In [3]:
with open('intents.json') as f:
    intents = json.load(f)

In [5]:
intents['intents'][1]['patterns'] = getBalanceData()
intents['intents'][2]['patterns'] = getBudgetingData()
intents['intents'][3]['patterns'] = getHousingData()

words = []
classes = []
documents = []

In [6]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w.isalnum()]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words")

1046 documents
7 classes ['affordability', 'balance', 'budgeting', 'exit', 'greeting', 'name', 'unknown']
1248 unique stemmed words


In [7]:
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(classes)

In [8]:
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])

In [9]:
tf.reset_default_graph()
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [10]:
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# Start training (apply gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 130999  | total loss: 0.00052 | time: 0.396s
| Adam | epoch: 1000 | loss: 0.00052 - acc: 1.0000 -- iter: 1040/1046
Training Step: 131000  | total loss: 0.00047 | time: 0.398s
| Adam | epoch: 1000 | loss: 0.00047 - acc: 1.0000 -- iter: 1046/1046
--
INFO:tensorflow:/Users/mehulshah/Desktop/financebot/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [11]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [12]:
p = bow("what is my bank account balance?", words)
print (p)
print (classes)

[0 0 0 ... 0 0 0]
['affordability', 'balance', 'budgeting', 'exit', 'greeting', 'name', 'unknown']


In [13]:
print(model.predict([p]))

[[2.8008789e-09 9.9999249e-01 7.5640942e-06 6.3683742e-17 1.0220878e-29
  1.2613123e-27 2.8859856e-18]]


In [14]:
# save all of our data structures# save  
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )